In [1]:
import numpy as np
import pandas as pd

import re
import string 
import contractions 

from tqdm import tqdm
tqdm.pandas(desc="Progress Bar")

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

/Users/zankhanamehta/Documents/NEU/NLP/Parse-Resume-main/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
jd_data = load_dataset('jacob-hugging-face/job-descriptions', split="train")
jd_data

Dataset({
    features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
    num_rows: 853
})

In [3]:
jd_df = pd.DataFrame(jd_data)
jd_df.head()

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo..."


In [4]:
print(jd_df['job_description'][0])

minimum qualifications
bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles
preferred qualifications
 years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills
about the job
as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organizations more product

In [5]:
df = pd.read_csv('data/extracted_resume.csv')
df.head()

,Skills,Education,ID,Category
0,Excellent classroom managementÂ,Subject Matter Authorization in Science: Scien...,37201447,AGRICULTURE
1,"Team mediation, Budget Management, Delegation ...","2009 Howard University ï¼​ City , State , USA ...",12674256,AGRICULTURE
2,"COMPUTER LITERACY, E-mail, English, government...","2011\nThe Universty of Zambia ï¼​ City , State...",29968330,AGRICULTURE
3,"C, C++, communication skills, designing, ELISA...","Masters of Science , Biotechnology 5 2013 Univ...",81042872,AGRICULTURE
4,"Data Entry, Printers, Clients, Loans, Tax Retu...",Wayne State University 2013 MBA : Linguistics ...,20006992,AGRICULTURE


In [ ]:
def text_cleaning(text:str) -> str:
    """
    Function to clean the text data
    
    Args:
    text (str): Text to be cleaned
    Returns:
    str: Cleaned text
    """

    if pd.isnull(text):
        return
    
    # lower-case everything
    text = text.lower().strip()
    
    # For removing puctuations
    translator = str.maketrans('', '', string.punctuation)
    
    # expand all the short-form words
    text = contractions.fix(text)
    
    # remove any special chars
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) # Remove URLs
    text = re.sub(r'\S+@\S+', '', text) # Remove emails
    text = re.sub(r'\b\d{1,3}[-./]?\d{1,3}[-./]?\d{1,4}\b', '', text) # Remove phone numbers
    text = text.translate(translator) # Remove puctuations
    text = re.sub(r'[^a-zA-Z]', ' ', text) # Remove other non-alphanumeric characters
    
    return text.strip()

In [ ]:
# We have 15 Resumes where Skills & Education were not extracted
# So, let's remove them
cv_df = df[~(df['Skills'].isna() & df['Education'].isna())].reset_index(drop=True)

# Filling the null values in Skills & Education with Empty String before concatinating them
cv_df = cv_df.fillna(value='')

# Let's stitch together Skills & Education, similar to given in job description.
cv_df['CV'] = cv_df['Skills'] + ' ' + cv_df['Education']

# Doing text cleaning
cv_df['CV'] = cv_df['CV'].progress_apply(text_cleaning)

Progress Bar: 100%|██████████| 2469/2469 [00:00<00:00, 18313.18it/s]


In [8]:
cv_df.shape

(2469, 5)

In [9]:
# Sample job descriptions
job_descriptions = jd_df['job_description'].apply(text_cleaning)[:15].to_list() # jd_df['job_description'][:15]

# Sample resumes (replace with your extracted resume data)
resumes = cv_df['CV'].to_list()

## Creating Embedding using `LLaMA1.1Tokenizer`, `LLaMA1.1Model`

In [10]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModel.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Function to get mean pooled embeddings
def get_llama_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Generate embeddings for job descriptions
job_description_embeddings = []
for description in job_descriptions:
    emb = get_llama_embedding(description)
    job_description_embeddings.append(emb)

# Generate embeddings for resumes
resume_embeddings = []
for resume in resumes:
    emb = get_llama_embedding(resume)
    resume_embeddings.append(emb)

/Users/zankhanamehta/Documents/NEU/NLP/Parse-Resume-main/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
job_description_embeddings[0].shape, resume_embeddings[0].shape

((2048,), (2048,))

In [12]:
len(job_description_embeddings), len(resume_embeddings)

(15, 2469)

## Calculating Similarity Score & Getting Top 5 Candidates

In [13]:
similarity_scores = cosine_similarity(job_description_embeddings, resume_embeddings)
similarity_scores

array([[0.73694205, 0.7323406 , 0.7253606 , ..., 0.63971424, 0.69180536,
        0.65899545],
       [0.7752715 , 0.7874105 , 0.748116  , ..., 0.63964283, 0.7316997 ,
        0.7305553 ],
       [0.73980534, 0.75016516, 0.7066609 , ..., 0.62804776, 0.69999486,
        0.6793622 ],
       ...,
       [0.7850693 , 0.7839699 , 0.7486244 , ..., 0.66037965, 0.72308266,
        0.7173516 ],
       [0.66914344, 0.7137781 , 0.66397667, ..., 0.6896409 , 0.70143676,
        0.58789814],
       [0.76395524, 0.7322281 , 0.7284547 , ..., 0.66410553, 0.7092798 ,
        0.6688111 ]], dtype=float32)

In [14]:
num_top_candidates = 10
top_candidates = []

for i, job_description in enumerate(job_descriptions):
    candidates_with_scores = list(enumerate(similarity_scores[i]))
    candidates_with_scores.sort(key=lambda x: x[1], reverse=True)
    top_candidates_for_job = candidates_with_scores[:num_top_candidates]
    top_candidates.append(top_candidates_for_job)

# Print the top candidates for each job description
for i, job_description in enumerate(job_descriptions):
    print(f"Top candidates for JD {i+1} - Postition: {jd_df['position_title'][i]}")
    for candidate_index, score in top_candidates[i]:
        print(f"  Candidate {candidate_index + 1} - Similarity Score: {score:.4f} - {cv_df['Category'][candidate_index]}/{cv_df['ID'][candidate_index]}.pdf")
        # print(f"  Resume: {resumes[candidate_index]}")
    print()

Top candidates for JD 1 - Postition: Sales Specialist
  Candidate 1049 - Similarity Score: 0.8513 - BANKING/39569771.pdf
  Candidate 227 - Similarity Score: 0.8502 - SALES/29184740.pdf
  Candidate 704 - Similarity Score: 0.8443 - HEALTHCARE/12315079.pdf
  Candidate 1932 - Similarity Score: 0.8426 - BUSINESS-DEVELOPMENT/20317319.pdf
  Candidate 1268 - Similarity Score: 0.8414 - INFORMATION-TECHNOLOGY/17111768.pdf
  Candidate 693 - Similarity Score: 0.8380 - HEALTHCARE/16121387.pdf
  Candidate 110 - Similarity Score: 0.8334 - ARTS/13272204.pdf
  Candidate 1943 - Similarity Score: 0.8323 - BUSINESS-DEVELOPMENT/67501448.pdf
  Candidate 1830 - Similarity Score: 0.8287 - FITNESS/24905947.pdf
  Candidate 387 - Similarity Score: 0.8281 - CONSULTANT/49777184.pdf

Top candidates for JD 2 - Postition: Apple Solutions Consultant
  Candidate 227 - Similarity Score: 0.8930 - SALES/29184740.pdf
  Candidate 521 - Similarity Score: 0.8828 - CHEF/16924102.pdf
  Candidate 200 - Similarity Score: 0.8817 -

In [15]:
selected_jd_indices = [0, 1, 2, 3, 4]

top_data = []

for jd_index in selected_jd_indices:
    # Top matching resume for this JD
    top_resume_index, top_score = top_candidates[jd_index][0]
    
    # Extract resume details
    resume_id = cv_df['ID'][top_resume_index]
    resume_cat = cv_df['Category'][top_resume_index]
    
    top_data.append({
        "JD Title": jd_df['position_title'][jd_index],
        "Resume ID": resume_id,
        "Resume Category": resume_cat,
        "Similarity Score": top_score
    })

top_resumes_df = pd.DataFrame(top_data)
top_resumes_df
# print(top_resumes_df)


,JD Title,Resume ID,Resume Category,Similarity Score
0,Sales Specialist,39569771,BANKING,0.851283
1,Apple Solutions Consultant,29184740,SALES,0.892980
2,Licensing Coordinator - Consumer Products,12237267,PUBLIC-RELATIONS,0.888181
3,Web Designer,26790545,DESIGNER,0.882117
4,Web Developer,62994611,AGRICULTURE,0.860130


Getting Top-10 Average Scores

In [16]:
avg_top10_scores = []

for jd_index in selected_jd_indices:
    top_10 = top_candidates[jd_index][:10]
    
    scores = [score for _, score in top_10]
    
    avg_score = sum(scores) / len(scores)
    
    avg_top10_scores.append({
        "JD Title": jd_df['position_title'][jd_index],
        "Average Top-10 Score": round(avg_score, 4)
    })

avg_top10_df = pd.DataFrame(avg_top10_scores)

print(avg_top10_df)


                                    JD Title  Average Top-10 Score
0                           Sales Specialist                0.8390
1                 Apple Solutions Consultant                0.8743
2  Licensing Coordinator - Consumer Products                0.8608
3                               Web Designer                0.8586
4                              Web Developer                0.8478


In [17]:
# Fixed selections
selected_jd_indices = [0, 1, 2, 3, 4]
selected_resume_indices = [1160, 1799, 2112, 1268, 1430, 1360, 713, 1944, 663, 1551]

# Initialize matrix
llama_matrix = []

for resume_idx in selected_resume_indices:
    row = []
    for jd_idx in selected_jd_indices:
        sim_score = similarity_scores[jd_idx][resume_idx]
        row.append(round(sim_score, 4))
    llama_matrix.append(row)

# Prepare column & row labels
jd_titles = [jd_df["position_title"][i] for i in selected_jd_indices]
resume_labels = [f"{cv_df['Category'][i]}/{cv_df['ID'][i]}" for i in selected_resume_indices]

# Create DataFrame
llama_df = pd.DataFrame(llama_matrix, index=resume_labels, columns=jd_titles)
llama_df

,Sales Specialist,Apple Solutions Consultant,Licensing Coordinator - Consumer Products,Web Designer,Web Developer
ACCOUNTANT/75286906,0.7349,0.7434,0.7302,0.7795,0.7562
FITNESS/27974588,0.4165,0.4703,0.4582,0.4724,0.4352
ADVOCATE/14445309,0.6623,0.6807,0.6581,0.7324,0.6824
INFORMATION-TECHNOLOGY/29975124,0.7626,0.7698,0.7327,0.7805,0.7434
CONSTRUCTION/30397268,0.6732,0.7236,0.6888,0.7620,0.7112
HR/22323967,0.7084,0.7173,0.7039,0.7492,0.7209
HEALTHCARE/24025053,0.7217,0.7557,0.7255,0.7716,0.7275
BUSINESS-DEVELOPMENT/12546838,0.3833,0.5215,0.4501,0.4676,0.3772
HEALTHCARE/23944036,0.6516,0.6975,0.6808,0.6905,0.6502
DESIGNER/25023614,0.7581,0.8282,0.7731,0.8205,0.7639


Llama, a model designed for lightweight and efficient natural language processing tasks, demonstrated relatively lower similarity scores compared to BERT and GPT-2. The results highlighted Llama’s inability to capture the subtle nuances and complex contextual relationships that were more effectively captured by BERT and GPT-2